In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt 
import spacy 
import math
import scipy
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn
import scipy

In [2]:
brunch_cafe = pd.read_pickle("C:/Users/61424/Desktop/MBusA/Module 5/Business Case Study/yelp_dataset/Competition/brunch_cafe.pkl")

In [65]:
health_take_away = pd.read_pickle("C:/Users/61424/Desktop/MBusA/Module 5/Business Case Study/yelp_dataset/Competition/health_take_away.pkl")

In [3]:
business_clean4 = pd.read_pickle("C:/Users/61424/Desktop/MBusA/Module 5/Business Case Study/yelp_dataset/Business/business_clean4.pkl")

In [5]:
brunch_cafe_busienss_id = brunch_cafe['business_id'].unique()

In [6]:
brunch_cafe = business_clean4[business_clean4['category_parent'] == ('breakfast & brunch' or 'cafes') ]

In [78]:
brunch_cafe.to_pickle("C:/Users/61424/Desktop/MBusA/Module 5/Business Case Study/yelp_dataset/Competition/brunch_cafe.pkl")

In [11]:
cafe_business = business_clean4[business_clean4['business_id'].apply(lambda x: x in cafe_busienss_id)]

In [31]:
#Explore country:
country_1 = pd.DataFrame(brunch_cafe.groupby('country').count()['business_id'])
country_1['percentage'] = country_1.business_id / sum(country_1.business_id)
country_1

,business_id,percentage
country,,
Canada,1027,0.524783
U.S,930,0.475217


In [13]:
#Explore country:
country_2 = pd.DataFrame(business_clean4.groupby('country').count()['business_id'])
country_2['percentage'] = country_2.business_id / sum(country_2.business_id)
country_2

,business_id,percentage
country,,
Canada,22659,0.399334
U.K,3,0.000053
U.S,34080,0.600613


#Interestingly, cafe business has a strong presence in Canada

In [59]:
#Compare State and city
state_coverage = len(brunch_cafe['state'].unique())/len(business_clean4['state'].unique())
print("cafe business has covered {} in all states".format(state_coverage))


#In U.S:
US_cafe = brunch_cafe[brunch_cafe['country'].apply(lambda x: x == 'U.S')]
US_business = business_clean4[business_clean4['country'].apply(lambda x: x == 'U.S')]

US_coverage = len(US_cafe['state'].unique())/len(US_cafe['state'].unique())
print("cafe business has covered {} in U.S states".format(US_coverage))

#In Canada:
CA_cafe = brunch_cafe[brunch_cafe['country'].apply(lambda x: x == 'Canada')]
CA_business = business_clean4[business_clean4['country'].apply(lambda x: x == 'Canada')]

CA_coverage = len(CA_cafe['state'].unique())/len(CA_business['state'].unique())
print("cafe business has covered {} in Canada states".format(CA_coverage))

location_1 = pd.DataFrame(brunch_cafe.groupby(['state','city']).count()['business_id']).sort_values(by=['business_id'], ascending = False)
location_1['percentage'] = location_1.business_id / sum(location_1.business_id)
location_1



cafe business has covered 0.4782608695652174 in all states
cafe business has covered 1.0 in U.S states
cafe business has covered 0.42857142857142855 in Canada states


,,business_id,percentage
state,city,,
ON,Toronto,336,0.171691
QC,MontrÃ©al,225,0.114972
NV,Las Vegas,160,0.081758
AZ,Phoenix,120,0.061318
AB,Calgary,106,0.054165
NC,Charlotte,76,0.038835
PA,Pittsburgh,64,0.032703
AZ,Scottsdale,64,0.032703
OH,Cleveland,48,0.024527


#Seemed cafe business really like Toronto and Montral - maybe because Canadian likes cafe!

In [34]:
location_3 = pd.DataFrame(brunch_cafe.groupby(['country','state']).count()['business_id']).sort_values(by=['business_id'], ascending = False)
location_3['percentage'] = location_3.business_id / sum(location_3.business_id)
location_3

business_id  percentage
country state                         
Canada  ON             539    0.275422
        QC             377    0.192642
U.S     AZ             329    0.168114
        NV             194    0.099131
        OH             122    0.062340
        NC             116    0.059274
Canada  AB             111    0.056719
U.S     PA              94    0.048033
        WI              54    0.027593
        IL              17    0.008687
        SC               4    0.002044

In [ ]:
#Compare business prescence 

In [62]:
location_0 = pd.DataFrame(business_clean4.groupby(['state','city']).count()['business_id']).sort_values(by=['business_id'], ascending = False)
location_0['percentage'] = location_0.business_id / sum(location_0.business_id)
location_0.reset_index(level = 0, inplace = True)
location_0.reset_index(level = 0, inplace = True)
location_0.columns = ['city','state','total_business','total_business_percentage']

In [63]:
#location_1.reset_index(level = 0, inplace = True)
#location_1.reset_index(level = 0, inplace = True)
location_1.columns = ['city','state','total_brunch_cafe','total_brunch_cafe_percentage']

In [66]:
location_5 = pd.DataFrame(health_take_away.groupby(['state','city']).count()['business_id']).sort_values(by=['business_id'], ascending = False)
location_5['percentage'] = location_5.business_id / sum(location_5.business_id)
location_5.reset_index(level = 0, inplace = True)
location_5.reset_index(level = 0, inplace = True)


In [69]:
location_5.columns = ['city','state','total_health_take_away','total_health_take_away_percentage']
#location_5

In [72]:
location_temp = location_0.merge(location_1,on = ['city','state',],how= 'left')

In [75]:
location_summary = location_temp.merge(location_5,on = ['city','state',],how= 'left')
location_summary = location_summary.fillna(0)
location_summary

,city,state,total_business,total_business_percentage,total_brunch_cafe,total_brunch_cafe_percentage,total_health_take_away,total_health_take_away_percentage
0,Toronto,ON,7682,0.135385,336.0,0.171691,2.0,0.010417
1,Las Vegas,NV,6058,0.106764,160.0,0.081758,11.0,0.057292
2,Phoenix,AZ,3805,0.067058,120.0,0.061318,12.0,0.062500
3,MontrÃ©al,QC,3632,0.064009,225.0,0.114972,0.0,0.000000
4,Calgary,AB,2767,0.048765,106.0,0.054165,0.0,0.000000
5,Charlotte,NC,2577,0.045416,76.0,0.038835,17.0,0.088542
6,Pittsburgh,PA,2360,0.041592,64.0,0.032703,17.0,0.088542
7,Scottsdale,AZ,1461,0.025748,64.0,0.032703,6.0,0.031250
8,Mississauga,ON,1410,0.024849,40.0,0.020439,3.0,0.015625
9,Cleveland,OH,1387,0.024444,48.0,0.024527,6.0,0.031250


In [22]:
#Popularity of category_parent

In [79]:
def get_bag_of_words (document):
    
    bag_of_words = {}
    
    #store the frequency of words:
    for line in document:
        for word in line:
            
            if word in bag_of_words:
                bag_of_words[word] +=1
            else:
                bag_of_words[word] = 1
                
    
    
    return bag_of_words

In [80]:
def two_gram_token(document):
    two_gram_list = []
    for line in document:
        length = len(line)
        i = 0
        two_gram_line = []
        
        while i < (length-1):
            two_gram = [line[i],line[i+1]]
            i += 1 
            two_gram_line.append(two_gram)
            
        two_gram_list.append(two_gram_line)
 
    return two_gram_list 

In [81]:
def get_grams_of_words (document):
    
    bag_of_words = {}
    
    #store the frequency of words:
    for line in document:
        for word in line:
            word = tuple(word)
            if word in bag_of_words:
                bag_of_words[word] +=1
            else:
                bag_of_words[word] = 1
                
    
    
    return bag_of_words

In [82]:
#To sort dictionary:
def sort_freq(mydict):
    sorted_dict = sorted(mydict.items(), key=lambda kv: kv[1], reverse=True)
    return sorted_dict

In [35]:
category_pop = pd.DataFrame(brunch_cafe.groupby('category_parent').count()['business_id']).sort_values('business_id', ascending = False)
category_pop['percentage'] = category_pop.business_id / sum(category_pop.business_id)
category_pop

,business_id,percentage
category_parent,,
breakfast & brunch,1957,1.0


In [85]:
brunch_cafe_two_gram = two_gram_token(brunch_cafe['category_token'])
brunch_cafe_two_gram_token_df = pd.DataFrame(sort_freq(get_grams_of_words(brunch_cafe_two_gram)))
brunch_cafe_two_gram_token_df

,0,1
0,"(Restaurants, Breakfast & Brunch)",548
1,"(Breakfast & Brunch, Restaurants)",519
2,"(Food, Restaurants)",162
3,"(Food, Breakfast & Brunch)",154
4,"(Breakfast & Brunch, Food)",153
5,"(Restaurants, Food)",147
6,"(Breakfast & Brunch, Coffee & Tea)",103
7,"(Coffee & Tea, Restaurants)",89
8,"(Food, Coffee & Tea)",88
9,"(Coffee & Tea, Food)",81


In [84]:
token_category = sort_freq(get_bag_of_words(brunch_cafe['category_token']))
token_category_df = pd.DataFrame(token_category)
token_category_df.columns = ['category_token','freq']
token_category_df

,category_token,freq
0,Restaurants,1957
1,Breakfast & Brunch,1957
2,Food,819
3,Coffee & Tea,446
4,Cafes,315
5,Diners,251
6,American (New),215
7,Bakeries,171
8,Canadian (New),151
9,Desserts,107


#Explore Business Characteristics 

In [11]:
business_attributes = pd.read_pickle("C:/Users/61424/Desktop/MBusA/Module 5/Business Case Study/yelp_dataset/business_attributes.pkl")

In [102]:
business_attributes['NoiseLevel'] = business_attributes['NoiseLevel'].apply(lambda x: str(x).replace('u',''))

In [12]:
part_attributes = ['business_id','RestaurantsReservations', 'Caters', 'NoiseLevel', 'RestaurantsTableService',
'RestaurantsTakeOut', 'RestaurantsPriceRange2', 'OutdoorSeating', 'GoodForKids', 'Alcohol',
'RestaurantsAttire', 'RestaurantsGoodForGroups', 'RestaurantsDelivery']
business_attributes_part = business_attributes.loc[:,part_attributes]

In [13]:
brunch_cafe_attributes = business_attributes_part[business_attributes_part['business_id'].apply(lambda  x: x in brunch_cafe_busienss_id)]

In [106]:
#RestaurantReservations
health_reservation = pd.DataFrame(brunch_cafe_attributes.groupby('RestaurantsReservations').count()['business_id'])
business_reservation = pd.DataFrame(business_attributes_part.groupby('RestaurantsReservations').count()['business_id'])
health_reservation['percentage'] = health_reservation.business_id / sum(health_reservation.business_id)
business_reservation['percentage'] = business_reservation.business_id / sum(business_reservation.business_id)

health_reservation

,business_id,percentage
RestaurantsReservations,,
False,406,0.750462
None,3,0.005545
True,132,0.243993


In [107]:
print(business_reservation)

                         business_id  percentage
RestaurantsReservations                         
False                          30807    0.611117
None                              43    0.000853
True                           19561    0.388030


#Most brunches and cafes dont require reservations

In [108]:
#Caters
health_caters = pd.DataFrame(brunch_cafe_attributes.groupby('Caters').count()['business_id'])
business_caters = pd.DataFrame(brunch_cafe_attributes.groupby('Caters').count()['business_id'])
health_caters['percentage'] = health_caters.business_id / sum(health_caters.business_id)
business_caters['percentage'] = business_caters.business_id / sum(business_caters.business_id)

health_caters


,business_id,percentage
Caters,,
False,161,0.474926
None,6,0.017699
True,172,0.507375


In [109]:
business_caters

,business_id,percentage
Caters,,
False,161,0.474926
None,6,0.017699
True,172,0.507375


In [110]:
#NoiseLevel
health_noise_level = pd.DataFrame(brunch_cafe_attributes.groupby('NoiseLevel').count()['business_id'])
business_noise_level = pd.DataFrame(business_attributes_part.groupby('NoiseLevel').count()['business_id'])
health_noise_level['percentage'] = health_noise_level.business_id / sum(health_noise_level.business_id)
business_noise_level['percentage'] = business_noise_level.business_id / sum(business_noise_level.business_id)

health_noise_level

,business_id,percentage
NoiseLevel,,
'average',244,0.343662
'lod',21,0.029577
'qiet',141,0.198592
'very_lod',9,0.012676
None,3,0.004225
nan,292,0.411268


In [111]:
business_noise_level

,business_id,percentage
NoiseLevel,,
'average',27005,0.454852
'lod',3369,0.056745
'qiet',9400,0.158326
'very_lod',1206,0.020313
None,66,0.001112
nan,18325,0.308652


In [112]:
#Overall, brunches' noise level are more quiet and less loud

In [113]:
#RestaurantsTableService
health_table_service = pd.DataFrame(brunch_cafe_attributes.groupby('RestaurantsTableService').count()['business_id'])
business_table_service = pd.DataFrame(business_attributes_part.groupby('RestaurantsTableService').count()['business_id'])
health_table_service['percentage'] = health_table_service.business_id / sum(health_table_service.business_id)
business_table_service['percentage'] = business_table_service.business_id / sum(business_table_service.business_id)

health_table_service

,business_id,percentage
RestaurantsTableService,,
False,102,0.528497
None,1,0.005181
True,90,0.466321


In [114]:
business_table_service

,business_id,percentage
RestaurantsTableService,,
False,6413,0.373870
None,29,0.001691
True,10711,0.624439


In [ ]:
#Brunches have a below industry average table service level but higher than health food takeaways (0.18)

In [115]:
#RestaurantsTakeOut
health_takeout = pd.DataFrame(brunch_cafe_attributes.groupby('RestaurantsTakeOut').count()['business_id'])
business_takeout = pd.DataFrame(business_attributes_part.groupby('RestaurantsTakeOut').count()['business_id'])
health_takeout['percentage'] = health_takeout.business_id / sum(health_takeout.business_id)
business_takeout['percentage'] = business_takeout.business_id / sum(business_takeout.business_id)

health_takeout

,business_id,percentage
RestaurantsTakeOut,,
False,40,0.066778
None,5,0.008347
True,554,0.924875


In [116]:
business_takeout

,business_id,percentage
RestaurantsTakeOut,,
False,3364,0.064306
None,52,0.000994
True,48896,0.934699


In [ ]:
#Most brunches support takeouts

In [117]:
#RestaurantsPriceRange2
health_pricerange = pd.DataFrame(brunch_cafe_attributes.groupby('RestaurantsPriceRange2').count()['business_id'])
business_pricerange = pd.DataFrame(business_attributes_part.groupby('RestaurantsPriceRange2').count()['business_id'])
health_pricerange['percentage'] = health_pricerange.business_id / sum(health_pricerange.business_id)
business_pricerange['percentage'] = business_pricerange.business_id / sum(business_pricerange.business_id)

health_pricerange

,business_id,percentage
RestaurantsPriceRange2,,
1,287,0.499130
2,265,0.460870
3,16,0.027826
4,4,0.006957
None,3,0.005217


In [119]:
business_pricerange

,business_id,percentage
RestaurantsPriceRange2,,
1,20914,0.401490
2,27786,0.533413
3,2837,0.054462
4,506,0.009714
None,48,0.000921


In [118]:
#Overall, brunches' price range similar to the industry average, higher than health take away fast food

,business_id,percentage
RestaurantsPriceRange2,,
1,20914,0.401490
2,27786,0.533413
3,2837,0.054462
4,506,0.009714
None,48,0.000921


In [120]:
#OutdoorSeating
health_outdoorseating = pd.DataFrame(brunch_cafe_attributes.groupby('OutdoorSeating').count()['business_id'])
business_outdoorseating = pd.DataFrame(business_attributes_part.groupby('OutdoorSeating').count()['business_id'])
health_outdoorseating['percentage'] = health_outdoorseating.business_id / sum(health_outdoorseating.business_id)
business_outdoorseating['percentage'] = business_outdoorseating.business_id / sum(business_outdoorseating.business_id)

health_outdoorseating

,business_id,percentage
OutdoorSeating,,
False,256,0.533333
None,3,0.006250
True,221,0.460417


In [121]:
business_outdoorseating

,business_id,percentage
OutdoorSeating,,
False,29356,0.607508
None,61,0.001262
True,18905,0.391230


#Brunches' outdoor sitting is similar to industry average, but a lot higher than healthy takeaways 

In [14]:
#Goodforgroups 
health_goodforgroups = pd.DataFrame(brunch_cafe_attributes.groupby('RestaurantsGoodForGroups').count()['business_id'])
business_goodforgroups = pd.DataFrame(business_attributes_part.groupby('RestaurantsGoodForGroups').count()['business_id'])
health_goodforgroups['percentage'] = health_goodforgroups.business_id / sum(health_goodforgroups.business_id)
business_goodforgroups['percentage'] = business_goodforgroups.business_id / sum(business_goodforgroups.business_id)

health_goodforgroups

,business_id,percentage
RestaurantsGoodForGroups,,
False,248,0.144691
True,1466,0.855309


In [15]:
#RestaurantsDelivery'
health_delivery = pd.DataFrame(brunch_cafe_attributes.groupby('RestaurantsDelivery').count()['business_id'])
business_delivery = pd.DataFrame(business_attributes_part.groupby('RestaurantsDelivery').count()['business_id'])
health_delivery['percentage'] = health_delivery.business_id / sum(health_delivery.business_id)
business_delivery['percentage'] = business_delivery.business_id / sum(business_delivery.business_id)

health_delivery

,business_id,percentage
RestaurantsDelivery,,
False,1500,0.878220
None,3,0.001756
True,205,0.120023


In [20]:
brunch_cafe_price = brunch_cafe_attributes.loc[:,['business_id','RestaurantsPriceRange2']]
brunch_cafe_price = brunch_cafe_price.dropna()


In [ ]:
#Data Export

In [77]:
location_summary.to_pickle("C:/Users/61424/Desktop/MBusA/Module 5/Business Case Study/yelp_dataset/Operation/location_summary.pkl")

In [22]:
brunch_cafe_price.to_csv("C:/Users/61424/Desktop/MBusA/Module 5/Business Case Study/yelp_dataset/Operation/brunch_cafe_price.csv")